In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

import kraft

In [ ]:
directory_path = "/home/kwat/garden/data/cancer_cell_line/copy_number/"

kraft.path.path(directory_path)

## Continuous

In [ ]:
gene_x_cell_line = pd.read_csv(
    "{}/CCLE_gene_cn.csv".format(directory_path), index_col=0
).T

gene_x_cell_line.index = (
    label.split()[0] for label in gene_x_cell_line.index.to_numpy()
)

gene_x_cell_line.columns = kraft.cell_line.rename(gene_x_cell_line.columns.to_numpy())

gene_x_cell_line.index.name = "Gene"

gene_x_cell_line.columns.name = "Cell Line"

kraft.dataframe.error_axes(gene_x_cell_line)

gene_x_cell_line.to_csv(
    "{}/gene_x_cell_line.tsv".format(directory_path), sep="\t",
)

gene_x_cell_line

## Binary

In [ ]:
mean = np.nanmean(gene_x_cell_line.to_numpy())

std = np.nanstd(gene_x_cell_line.to_numpy())

loss = mean - std * 3

gain = mean + std * 3

print(loss, gain)


def make_gene_loss_gain_x_(copy_numbers, loss, gain):

    gene_loss_gain_x_ = np.full((2, copy_numbers.size), 0)

    for i, copy_number in enumerate(copy_numbers.to_numpy()):

        if copy_number <= loss:

            gene_loss_gain_x_[0, i] = 1

        elif gain <= copy_number:

            gene_loss_gain_x_[1, i] = 1

    has_1 = (gene_loss_gain_x_ == 1).any(axis=1)

    gene = copy_numbers.name

    return pd.DataFrame(
        gene_loss_gain_x_[has_1],
        index=np.asarray(("{} loss".format(gene), "{} gain".format(gene)))[has_1],
        columns=copy_numbers.index,
    )

In [ ]:
file_path = "{}/gene_loss_gain_x_cell_line.tsv".format(directory_path)

n = gene_x_cell_line.shape[0]

n_per_print = n // 10

with open(file_path, mode="w") as io:

    for i, (_, copy_numbers) in enumerate(gene_x_cell_line.iterrows()):

        if i % n_per_print == 0:

            print("{}/{}...".format(i + 1, n))

        make_gene_loss_gain_x_(copy_numbers, loss, gain).to_csv(
            io, sep="\t", header=i == 0, mode="a"
        )

gene_loss_gain_x_cell_line = pd.read_csv(file_path, sep="\t", index_col=0)

gene_loss_gain_x_cell_line